# 1. Import dependencies

In [ ]:
import copy
import cv2 as cv
import csv
import mediapipe as mp
import numpy as np
import itertools
import os

from collections import deque

# 2. Initialization

## a. Initialize variables and models

In [ ]:
# Initialize Mediapipe's hand model 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=True,   
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5,
)

# Initialize main directory 
path = os.getcwd()
main_directory = os.path.dirname(path)

## b. Initialize functions

In [ ]:
# Calculate landmark list 
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [ ]:
# Pre-processing calculated landmarks 
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

## c. Initialize dataset template function

In [ ]:
# Initialize number of hand keypoints and alphabet class
num_coords = 21
alphabet = "A"

In [ ]:
# Initialize .csv file
def init_csv(feature):
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]

    with open(main_directory +'/dataset/csv_pre_combined/keypoints_{}.csv'.format(feature), mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

## d. Initialize dataset logging function

In [ ]:
def logging_csv(feature, landmark_list):

    csv_path = main_directory + '/dataset/csv_pre_combined/keypoints_{}.csv'.format(feature)
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([feature, *landmark_list])

    return

## e. Initialize image location

In [ ]:
IMAGE_FILES = []

# 3. Capture, re-normalize & write landmark keypoints into dataset

In [ ]:
# While in capturing process ##################################################
while True:

    # Application stops when "ESC" key is pressed
    if cv.waitKey(5) & 0xFF == 27:  # ESC key.
        break

    for idx, file in enumerate(IMAGE_FILES):
        image = cv.flip(cv.imread(file), 1)
        results = hands.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        
        if not results.multi_hand_landmarks:
            continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        
        cv.imwrite('/tmp/annotated_image' + str(idx) + '.png', cv.flip(annotated_imagen, 1))
        
        if not results.multi_hand_landmarks:
            continue
        
        for hand_world_landmars in results.multi_hand_landmarks:
            mp_drawing.plot_landmarks(
                hand_world_landmars, mp_hands.HAND_CONNECTIONS, azimuth=5)
cap.release()
    